In [ ]:
# fashion-mnist 데이터셋으로 cnn 모델 개발

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras as tf_keras

In [2]:
# 데이터 준비
(X_train, y_train), (X_test, y_test) = tf_keras.datasets.fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
# 데이터 확인

# label_name
# 0 T-shirt/top, 1 Trouser, 2 Pullover, 3 Dress, 4 Coat, 5 Sandal, 6 Shirt, 7 Sneaker, 8 Bag, 9 Ankle boot

print(X_train.shape, y_train.shape)
print(np.unique(y_train, return_counts=True))

(60000, 28, 28) (60000,)
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8), array([6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000]))


In [4]:
X_train.max()

255

In [5]:
# 데이터 전처리

X_train_scaled = X_train / 255 # X_train.max()
X_test_scaled = X_test / 255 # X_test.max()

print(X_train_scaled.max(), X_test_scaled.max())

1.0 1.0


In [10]:
# 모델 구조 설계 1

model = tf_keras.models.Sequential()
model.add(tf_keras.layers.Input(shape=(28, 28, 1)))
model.add(tf_keras.layers.Conv2D(filters=32,
                                 kernel_size=3, # 3 x 3 크기의 필터 사용
                                 padding='same', # zero padding을 적용해서 입력 이미지의 크기와 출력 피처맵의 크기를 같게 설정
                                 activation='relu'))
model.add(tf_keras.layers.MaxPool2D(pool_size=2))
model.add(tf_keras.layers.Conv2D(filters=64,
                                 kernel_size=3,
                                 padding='same',
                                 activation='relu'))
model.add(tf_keras.layers.MaxPool2D(pool_size=2))

model.add(tf_keras.layers.Flatten())

model.add(tf_keras.layers.Dense(units=128, activation='relu'))
model.add(tf_keras.layers.Dense(units=10, activation='softmax'))

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)                    │ (None, 28, 28, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 14, 14, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 7, 7, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 3136)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │         401,536 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 421,642 (1.61 MB)

 Trainable params: 421,642 (1.61 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# 모델 구조 설계 2 : 함수형 API 사용

input = tf_keras.layers.Input(shape=(28, 28, 1))
conv2d_1 = tf_keras.layers.Conv2D(filters=32,
                                  kernel_size=3, # 3 x 3 크기의 필터 사용
                                  padding='same', # zero padding을 적용해서 입력 이미지의 크기와 출력 피처맵의 크기를 같게 설정
                                  activation='relu')
maxpool2d_1 = tf_keras.layers.MaxPool2D(pool_size=2)
conv2d_2 = tf_keras.layers.Conv2D(filters=64,
                                  kernel_size=3,
                                  padding='same',
                                  activation='relu')
maxpool2d_2 = tf_keras.layers.MaxPool2D(pool_size=2)
flatten = tf_keras.layers.Flatten()
dense_1 = tf_keras.layers.Dense(units=128, activation='relu')
# dropout
dropout = tf_keras.layers.Dropout(rate=0.2)
output = tf_keras.layers.Dense(units=10, activation='softmax')

# conv2d_1(input) --> conv2d_1.__call__(input)
output2 = conv2d_1(input)
output2 = maxpool2d_1(output2)
output2 = conv2d_2(output2)
output2 = maxpool2d_2(output2)
output2 = flatten(output2)
output2 = dense_1(output2)
# dropout
output2 = output(output2)

model2 = tf_keras.models.Model(input, output2)
model2.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 28, 28, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 14, 14, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 7, 7, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 3136)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         401,536 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 421,642 (1.61 MB)

 Trainable params: 421,642 (1.61 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# 모델 학습 설계

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model2.compile(loss='sparse_categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'])

In [12]:
# 모델 학습 (훈련)

fit_history = model2.fit(X_train_scaled, y_train, batch_size=64, epochs=20, validation_split=0.2)

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 67s 87ms/step - accuracy: 0.7670 - loss: 0.6567 - val_accuracy: 0.8857 - val_loss: 0.3231
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 81s 86ms/step - accuracy: 0.8903 - loss: 0.3022 - val_accuracy: 0.8899 - val_loss: 0.2955
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 82s 86ms/step - accuracy: 0.9064 - loss: 0.2591 - val_accuracy: 0.9032 - val_loss: 0.2643
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 82s 86ms/step - accuracy: 0.9174 - loss: 0.2234 - val_accuracy: 0.9071 - val_loss: 0.2558
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 82s 86ms/step - accuracy: 0.9295 - loss: 0.1945 - val_accuracy: 0.9144 - val_loss: 0.2307
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 63s 84ms/step - accuracy: 0.9387 - loss: 0.1677 - val_accuracy: 0.9097 - val_loss: 0.2481
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 67s 90ms/step - accuracy: 0.9462 - loss: 0.1453 - val_accuracy: 0.9186 - val_loss: 0.2330
Epoch 8/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 80s 87ms/step - accuracy: 0.9541 - loss: 0.1249 - 

In [14]:
model2.evaluate(X_train_scaled, y_train), model2.evaluate(X_test_scaled, y_test)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 26s 14ms/step - accuracy: 0.9940 - loss: 0.0234
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9131 - loss: 0.4619


([0.09628584235906601, 0.9800500273704529],
 [0.43960845470428467, 0.9135000109672546])